# POI Categories - Google Places to Ontology System Paper

In [1]:
import os

os.chdir("/home/tales/dev/master/mdc_analysis/")
print("working dir", os.getcwd())

import pandas as pd
import matplotlib.pyplot as plt
import json

from src.dao import csv_dao
from src.entity.stop_region import StopRegionGroup
from src.poi_grabber import google_places

working dir /home/tales/dev/master/mdc_analysis


In [2]:
def remove_words_from_list(lista, words_to_remove=["establishment", "point_of_interest"]):
    for word_to_remove in words_to_remove:
        if word_to_remove in lista:
            lista.remove(word_to_remove)
    return lista

### Loading data

All POIs

In [3]:
all_pois = google_places.load_all_google_places_data(valid_pois=True)
pois_lite = all_pois[["name", "types"]]
pois_lite.head()

,name,types
place_id,,
ChIJTcpEIDQujEcRfRIRizbvj4k,Office d'impôt des districts de Lausanne et Ou...,"[accounting, finance, point_of_interest, local..."
ChIJPV-d_9dkjEcRh-AFXijs5H0,Graphiste Imprimeur Genève Agence BESTRIBUTION,"[store, point_of_interest, establishment]"
ChIJ6TtT3J1EjEcRdmsz6L2gHTU,institut beauté lys,"[beauty_salon, point_of_interest, establishment]"
ChIJLZi4AC8ujEcRvSGJiRYVAk0,cabinet evexia | coaching & soins thérapeutiques,"[health, point_of_interest, establishment]"
ChIJKTbDs6ygmkcRhp_reVwKcjc,RTM Financial Coaching AG,"[finance, point_of_interest, establishment]"


All visited POIs

In [4]:
users_sr = {}

users = os.listdir("outputs/stop_regions/")

for user_id in users:
    print("Loading user {} data".format(user_id))
    users_sr[user_id] = StopRegionGroup(csv_dao.stop_region_sequence(user_id), 
                                                    agglutinate_stop_regions=False)

Loading user 6189 data


/home/tales/dev/master/mdc_analysis/src/entity/stop_region.py:69: FutureWarning: 'place_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  self.close_pois = google_places.load_all_google_places_data(valid_pois=True).merge(pois, on="place_id", how="inner").sort_values(by="distance")


Loading user 5936 data
Loading user 6087 data
Loading user 5973 data
Loading user 6085 data
Loading user 6074 data
Loading user 6012 data
Loading user 5982 data
Loading user 5948 data
Loading user 5974 data
Loading user 6090 data
Loading user 6199 data
Loading user 6068 data
Loading user 6024 data
Loading user 5976 data
Loading user 6094 data
Loading user 5941 data
Loading user 5995 data
Loading user 5962 data
Loading user 6093 data
Loading user 6033 data
Loading user 6079 data
Loading user 6038 data
Loading user 6175 data
Loading user 6042 data
Loading user 5924 data
Loading user 6083 data
Loading user 6178 data
Loading user 5958 data
Loading user 6086 data
Loading user 6100 data
Loading user 5945 data
Loading user 5925 data
Loading user 5935 data
Loading user 6172 data
Loading user 5955 data
Loading user 6073 data
Loading user 5980 data
Loading user 6010 data
Loading user 5993 data
Loading user 6037 data
Loading user 5979 data
Loading user 5966 data
Loading user 5985 data
Loading use

New Ontology Category System Types

In [5]:
with open('../poi_catogories/data/categ_map.json', "r") as json_file:
    categ_map = json.load(json_file)

Merging data

In [6]:
def map_types(google_poi):
    print("Name:", google_poi["name"])
    google_types = remove_words_from_list(google_poi["types"])
    print("Types:", google_poi["types"])
    print()
    for a_type in google_types:
        print(">>>", categ_map[a_type])
    print("-----------")
    print()

In [7]:
pois_lite.head().apply(map_types, axis=1)

Name: Office d'impôt des districts de Lausanne et Ouest Lausannois
Types: ['accounting', 'finance', 'local_government_office']

>>> ['Business buildings', 'Companies & Enterprises', 'Financial and insurance']
>>> ['Business buildings', 'Companies & Enterprises', 'Financial and insurance']
>>> ['Government &  social groups']
-----------

Name: Graphiste Imprimeur Genève Agence BESTRIBUTION
Types: ['store']

>>> ['Business buildings', 'Companies & Enterprises', 'Shopping']
-----------

Name: institut beauté lys
Types: ['beauty_salon']

>>> ['Life service']
-----------

Name: cabinet evexia | coaching & soins thérapeutiques
Types: ['health']

>>> ['Medical Care']
-----------

Name: RTM Financial Coaching AG
Types: ['finance']

>>> ['Business buildings', 'Companies & Enterprises', 'Financial and insurance']
-----------



place_id
ChIJTcpEIDQujEcRfRIRizbvj4k    None
ChIJPV-d_9dkjEcRh-AFXijs5H0    None
ChIJ6TtT3J1EjEcRdmsz6L2gHTU    None
ChIJLZi4AC8ujEcRvSGJiRYVAk0    None
ChIJKTbDs6ygmkcRhp_reVwKcjc    None
dtype: object

## Types

### diversity

In [ ]:
types_list = pd.Series(d["types"].sum())
types_list = types_list[(types_list != "establishment") & (types_list != "point_of_interest")]
types_list = types_list.apply(lambda value : pd.Series(value).drop_duplicates().tolist()[0])

In [ ]:
f, ax = plt.subplots(figsize=(15,5))
types_list.astype(str).value_counts().head(35).plot.bar(title="Frequency of POI types")

### size

In [ ]:
types_list.head()

In [ ]:
d["types"].apply(len).describe()

In [ ]:
d["types"].apply(len).plot.hist(bins=10)

#### Types of similar places

In [ ]:
d[d["name"] == "McDonald’s"]["types"].astype(str).value_counts()

## Useful functions

In [ ]:
def term_list_index(term, lista):
    if term in lista:
        return lista.index(term)
    else:
        return -1

def term_index_len(term, types_series):
    term_index = types_series.apply(lambda lista : term_list_index(term, lista))
    term_types_len = types_series.apply(lambda lista : len(lista))

    return pd.DataFrame({"index": term_index, "len": term_types_len})

def left_right_term(term, types_series):
    lefts = []
    rights = []

    term_index_len_df = term_index_len(term, types_series)

    for type_row in types_series.loc[term_index_len_df[term_index_len_df["index"] >= 0].index]:
        left = type_row[0:type_row.index(term)]
        lefts = lefts + left
        right = type_row[type_row.index(term) + 1:]
        rights = rights + right

    lefts = pd.Series(lefts).rename({0: "left"})
    rights = pd.Series(rights).rename({0: "right"})

    lr = lefts.value_counts().to_frame().merge(rights.value_counts().to_frame(), how="outer",
                                                 left_index=True,
                                                 right_index=True)
    
    return lr.sort_values(by=["0_x", "0_y"], ascending=False).rename({"0_x": "left", "0_y": "right"}, axis=1)

def term_placement_analisis(lr, show=True):
    right = lr[(lr["left"].isna()) & (~lr["right"].isna())]
    left = lr[(lr["right"].isna()) & (~lr["left"].isna())]
    both_valid = lr[~(lr["right"].isna()) & (~lr["left"].isna())]
    both_nan = lr[(lr["right"].isna()) & (lr["left"].isna())]
    
    if show:
    
        print("RIGHT side occurrences")
        print()
        print(right)
        print("---")
        print()
        print("LEFT side occurrences")
        print(left)
        print("---")
        print()
        print("BOTH sides occurrences")
        print(both_valid)
        print("---")
        print()
    return {"right": right, "left": left, "both": both_valid}

# Searching for hierarchy patterns

In [ ]:
n_cases = d["types"].apply(lambda lista : "health" in lista).value_counts()
print("'health' happens in {:10.2f}% of cases".format(n_cases[True] * 100 / n_cases.sum()))

In [ ]:
def term_list_index(term, lista):
    if term in lista:
        return lista.index(term)
    else:
        return -1

In [ ]:
health_index = d["types"].apply(lambda lista : term_list_index('health', lista))
health_types_len = d["types"].apply(lambda lista : len(lista))

health = pd.DataFrame({"index": health_index, "len": health_types_len})

#### Index occurrence in types list and types list length

In [ ]:
health[health["index"] >= 0].sample(6)

In [ ]:
health[health["index"] >= 0].plot.scatter("len", "index", title="'health' term index vs types_list length")

In [ ]:
health.loc[health[(health["len"] >= 8) & (health["index"] >= 0)].index].sample(6)

In [ ]:
health_lr = left_right_term('health', d["types"])
lr = term_placement_analisis(health_lr)

### left vs right: term ocurrences 

In [ ]:
# unique_types_list = pd.Series(types.sum()).drop_duplicates()

In [ ]:
# types_list = pd.Series(types.sum())

In [ ]:
# types_list[(types_list != 'establishment') & (types_list != 'point_of_interest')].value_counts()

In [ ]:
# term = unique_types_list.sample().item()
# term

In [ ]:
term = "school"

print("Occurrences for term: {}".format(term))
print("\n\n")
lr_health = left_right_term(term, d["types"])
lr = term_placement_analisis(lr_health)

In [ ]:
term = "bar"

print("Occurrences for term: {}".format(term))
print("\n\n")
lr_health = left_right_term(term, d["types"])
lr = term_placement_analisis(lr_health)

In [ ]:
term = "store"

print("Occurrences for term: {}".format(term))
print("\n\n")
lr_health = left_right_term(term, d["types"])
lr = term_placement_analisis(lr_health)

In [ ]:
term = "hair_care"

print("Occurrences for term: {}".format(term))
print("\n\n")
lr_health = left_right_term(term, d["types"])
lr = term_placement_analisis(lr_health)

In [ ]:
term = "natural_feature"

print("Occurrences for term: {}".format(term))
print("\n\n")
lr_health = left_right_term(term, d["types"])
lr = term_placement_analisis(lr_health)

# Type popularity vs right / left

In [ ]:
types_list_freq = types_list[(types_list != 'establishment') & (types_list != 'point_of_interest')].value_counts()
types_list_freq = types_list_freq.to_frame().rename(columns={0: "freq_index"})
types_list_freq = types_list_freq.sort_values(by="freq_index", ascending=False).reset_index().rename(columns={"index": "term"}).reset_index()
types_list_freq.head()

In [ ]:
r_prop = []

for term in types_list_freq["term"]:
    lr = term_placement_analisis(left_right_term(term, d["types"]), show=False)
    r_prop.append({"term": term, "r_prop": len(lr["right"]) / (len(lr["left"]) + len(lr["right"]))})
    
r_prop = pd.DataFrame(r_prop).sort_values(by="r_prop", ascending=False)
r_prop = r_prop.sort_values(by="r_prop", ascending=False).reset_index(drop=True).reset_index()
r_prop.tail()

In [ ]:
r_prop_vs_freq_index = r_prop.merge(types_list_freq, left_on="term", right_on="term", how="inner").rename(columns={"index_x": "index_r_prop", "index_y": "index_freq"})
r_prop_vs_freq_index.head()

In [ ]:
r_prop_vs_freq_index["index_r_prop"].corr(r_prop_vs_freq_index["index_freq"], method="spearman")

In [ ]:
r_prop_vs_freq_index.plot.scatter(x="index_r_prop", y="index_freq", title="Scatter plot")

The more frequent the term is, the less right side elements it has

# Analising leftmosts categories

In [ ]:
def remove_words_from_list(lista, words_to_remove=["establishment", "point_of_interest"]):
    for word_to_remove in words_to_remove:
        if word_to_remove in lista:
            lista.remove(word_to_remove)
    return lista

In [ ]:
clean_types = d["types"].apply(lambda value : remove_words_from_list(value))
leftmost = clean_types[clean_types.astype(str) != "[]"].apply(lambda value : value[-1])

print(len(leftmost.drop_duplicates()), "leftmost categories")

# <font color="red">Ontology-based category system</font>

In [ ]:
types_list_unique = pd.Series(types_list).drop_duplicates()

In [ ]:
len(types_list_unique)

In [ ]:
def to_dict(lista):
    num_dict = {}
    for i in range(len(lista)):
        num_dict[str(i)] = lista[i]
    return num_dict

def pretty_print_categ_dict(categ_dict):
    for key in range(len(categ_dict.keys())):
        print("{}: {}".format(key, categ_dict[str(key)]))

In [ ]:
with open('../poi_catogories/data/poi_categories.json', 'r') as json_file:
    categories = sorted(json.load(json_file))

In [ ]:
categ_dict = to_dict(categories)
categ_dict

### Mapping Types Categories 

In [ ]:
categ_map = {}

In [ ]:
# for a_type in types_list_unique:
#     pretty_print_categ_dict(categ_dict)
#     print()
#     print(">>> {}".format(a_type))
#     if a_type in categ_map.keys():
#         pass
    
#     else:
#         categ_keys = input("").split(",")
        
#         if categ_keys == ['']:
#             categ_values = ""
#         else:
#             categ_values = []
#             for categ_key in categ_keys:
#                 categ_values.append(categ_dict[categ_key])

#         categ_map[a_type] = categ_values
#     print(a_type, ">>>", categ_map[a_type])
#     print()
#     print("--------------")

In [ ]:
# with open('../poi_catogories/data/categ_map.json', 'w') as outfile:
#     json.dump(categ_map, outfile)

In [ ]:
with open('../poi_catogories/data/categ_map.json', "r") as json_file:
    categ_map = json.load(json_file)


In [ ]:
def dict_to_df(a_dict, keys_colname, values_colname):
    rows = []
    for key in a_dict.keys():
        rows.append({keys_colname: key, values_colname: a_dict[key]})
    return pd.DataFrame(rows)

In [ ]:
categ_df = dict_to_df(categ_map, "google_type", "new_types")
categ_df.to_csv('../poi_catogories/data/categ_map.csv', index=False)
categ_df

# <font color="red">Ontology-based category system - Analysis</font>

In [ ]:
def expand_new_types(categ_df):
    expanded_rows = []
    
    for index, type_row in categ_df.iterrows():
        if len(type_row["new_types"]) == 0:
            expanded_rows.append({"google_type": type_row["google_type"],
                                  "new_type": ""})
            
        else:
            for new_type in type_row["new_types"]:
                expanded_rows.append({"google_type": type_row["google_type"],
                                      "new_type": new_type})
    
    return pd.DataFrame(expanded_rows)

expanded_categ_df = expand_new_types(categ_df)

## <font color="red">Empty Categories</font>

In [ ]:
expanded_categ_df[expanded_categ_df["new_type"] == '']